# RNA-seq Pipeline

Notice: Please open the notebook under /notebooks/awsCluster/BasicCFNClusterSetup.ipynb to install CFNCluster package on your Jupyter-notebook server before running the notebook.

# 1. Configure AWS key pair, data location on S3 and the project information

In [ ]:
import os
import sys

## S3 input and output addresses.
# Notice: DO NOT put a forward slash at the end of your addresses.
s3_input_files_address = "path/to/s3/input/address"
s3_output_files_address = "path/to/s3/output/address"

## Directory for this notebook - for the use of displaying multiqc report
notebook_dir = "path/to/this/notebook"

## Path to the design file
design_file = "path/to/design_file.txt"

## CFNCluster name
your_cluster_name = "cluster_name"

## The private key pair for accessing cluster.
private_key = "path/to/private_key"

## Project information
# Recommended: Specify year, month, date, user name and pipeline name (no empty spaces)
project_name = ""

## Choose from one of the workflows: "star_gatk", "star_htseq", "star_rsem"
workflow = ""

## Genome information: currently available genomes: hg19, mm10 (mm10 only supported in star_rsem as of now)
genome = ""

## Analysis steps:

# for star_gatk: "fastqc", "trim", "align", "variant_calling"
# for star_htseq: "fastqc", "trim", "align", "count", "merge"
# for star_rsem: "fastqc", "trim", "calculate_expression", "summary"

## Optional steps: 
# "multiqc": as the last step

analysis_steps = {""}

## If delete cfncluster after job is done.
delete_cfncluster = False

print ("Variables set.")


# 2. Create CFNCluster

Notice: The CFNCluster package can be only installed on Linux box which supports pip installation.

In [ ]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
                                              username="ec2-user",
                                              private_key_file=private_key)

# 3. Run the RNA sequencing pipeline

In [ ]:
from util import PipelineManager
from util import DesignFileLoader

## DO NOT EDIT BELOW
analysis_steps = {workflow + "_" + x for x in analysis_steps}
print (analysis_steps)

sample_list, group_list, pair_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("RNASeq", ssh_client, project_name, workflow, analysis_steps, s3_input_files_address,
                        sample_list, group_list, s3_output_files_address, genome, "NA", pair_list)

# 4. Display MultiQC report

### Note: Run the cells below after all jobs are done on the cluster.

In [ ]:
# Download the multiqc html file to local
!aws s3 cp $s3_output_files_address/$project_name/$workflow/multiqc_report.html $notebook_dir

In [ ]:
from IPython.display import IFrame
  
path_to_report = os.path.relpath(notebook_dir + "/multiqc_report.html")

IFrame(path_to_report, width="100%", height=1000)